In [6]:
import pickle

from dotenv import load_dotenv
import dspy

load_dotenv("../../.env")
lm = dspy.LM("gemini/gemini-2.5-pro", temperature=0.9, cache=False)
dspy.settings.configure(lm=lm, track_usage=True)

In [7]:
# Load the object from the pickle file
with open(
    "eval_summarized_tool_results.pkl",
    "rb",
) as f:
    eval_summarized_tool_results = pickle.load(f)

# Display the loaded object
len(eval_summarized_tool_results)

125

In [8]:
class SummarizeRun(dspy.Signature):
    """Produce a concise MARKDOWN of the run. Summarize information where possible to make it more concise, but never lose information.

    FORMAT:
    ## Compound
    <compound_name>

    ## Trajectory
    - Step 0:
        - <reasoning>
        - <tool>
        - <observation>
    - Step 1: ...

    ## Reasoning
    <reasoning text>

    ## Predicted Efficacy
    <predicted_efficacy>

    ## Confidence
    <confidence>
    """

    compound_name: str = dspy.InputField(
        desc="The name of the compound being evaluated"
    )
    trajectory: str = dspy.InputField(desc="The raw trace data to summarize")
    reasoning: str = dspy.InputField(
        desc="The reasoning used to arrive at the trace conclusion."
    )
    predicted_efficacy: float = dspy.InputField(
        desc="The predicted efficacy of the compound."
    )
    confidence: float = dspy.InputField(desc="The confidence level prediction.")
    summary: str = dspy.OutputField(
        desc="A concise, structured summary of key information formatted for use by ReAct agent LLM"
    )


summarizer_lm = dspy.LM(
    "gemini/gemini-2.5-flash-lite", temperature=0.0, cache=True, max_tokens=25000
)


class ReflectRun(dspy.Signature):
    """Calculate the error of the run as the absolute difference between the predicted efficacy and the real efficacy.
    Then, produce a thorough reflection as to why the error occurred and how to improve the prediction accuracy in future runs.

    FORMAT:
    ## Real Efficacy
    <real efficacy>

    ## Error
    <|predicted efficacy - real efficacy|>

    ## Error Reasoning
    <error reasoning text>

    ## Improvements
        1) <improvement 1>
        2) <improvement 2>
        3) ...
    """

    summarized_run: str = dspy.InputField(desc="The the run being evaluated")
    real_efficacy: float = dspy.InputField(desc="The actual efficacy of the compound.")
    reflection: str = dspy.OutputField()


reflection_lm = dspy.LM(
    "gemini/gemini-2.5-pro", temperature=0.5, cache=True, max_tokens=25000
)

result_num = 0
docs = []
for result in eval_summarized_tool_results:
    compound_name = result[0]["compound_name"]
    efficacy = round(result[0]["cf_efficacy"], 2)
    run = result[1]

    print("Processing result", result_num, "for compound", compound_name)

    with dspy.context(lm=summarizer_lm):
        summary_predict = dspy.Predict(SummarizeRun)
        summary_result = summary_predict(
            compound_name=compound_name,
            trajectory=run.trajectory,
            reasoning=run.reasoning,
            predicted_efficacy=run.predicted_efficacy,
            confidence=run.confidence,
        )

    with dspy.context(lm=reflection_lm):
        reflection_predict = dspy.Predict(ReflectRun)
        reflection_result = reflection_predict(
            summarized_run=summary_result.summary,
            real_efficacy=efficacy,
        )

    doc = f"""# Agent Run Summary
{summary_result.summary}

# Accuracy Reflection
{reflection_result.reflection}"""

    docs.append(doc)
    print("Finished result", result_num)

    result_num += 1

len(docs)

Processing result 0 for compound Luminespib
Finished result 0
Processing result 1 for compound Tanespimycin
Finished result 1
Processing result 2 for compound MK-2866
Finished result 2
Processing result 3 for compound Navitoclax
Finished result 3
Processing result 4 for compound Finasteride


/Users/roshankern/Desktop/Github/cf-compound-selection-demo/backend/.venv/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content='[[ ## re...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


Finished result 4
Processing result 5 for compound Veliparib


/Users/roshankern/Desktop/Github/cf-compound-selection-demo/backend/.venv/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content='[[ ## su...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


Finished result 5
Processing result 6 for compound GSK1904529A


...

/Users/roshankern/Desktop/Github/cf-compound-selection-demo/backend/.venv/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content='## Compo...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
/Users/roshankern/Desktop/Github/cf-compound-selection-demo/backend/.venv/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content='{\n  "su...er_specific_fields=None), input

Finished result 14
Processing result 15 for compound Varespladib
Finished result 15
Processing result 16 for compound Temsirolimus
Finished result 15
Processing result 16 for compound Temsirolimus
Finished result 16
Processing result 17 for compound Ridaforolimus
Finished result 16
Processing result 17 for compound Ridaforolimus
Finished result 17
Processing result 18 for compound SU11274
Finished result 17
Processing result 18 for compound SU11274
Finished result 18
Processing result 19 for compound Dutasteride
Finished result 18
Processing result 19 for compound Dutasteride
Finished result 19
Processing result 20 for compound Fluvoxamine maleate
Finished result 19
Processing result 20 for compound Fluvoxamine maleate
Finished result 20
Processing result 21 for compound PF-04217903
Finished result 20
Processing result 21 for compound PF-04217903
Finished result 21
Processing result 22 for compound Etomidate
Finished result 21
Processing result 22 for compound Etomidate
Finished result

/Users/roshankern/Desktop/Github/cf-compound-selection-demo/backend/.venv/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content="## Compo...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


Finished result 44
Processing result 45 for compound Fluvoxamine maleate
Finished result 45
Processing result 46 for compound PF-04217903
Finished result 45
Processing result 46 for compound PF-04217903
Finished result 46
Processing result 47 for compound Etomidate
Finished result 46
Processing result 47 for compound Etomidate
Finished result 47
Processing result 48 for compound PD184352
Finished result 47
Processing result 48 for compound PD184352
Finished result 48
Processing result 49 for compound Everolimus
Finished result 48
Processing result 49 for compound Everolimus
Finished result 49
Processing result 50 for compound Luminespib
Finished result 49
Processing result 50 for compound Luminespib
Finished result 50
Processing result 51 for compound Tanespimycin
Finished result 50
Processing result 51 for compound Tanespimycin
Finished result 51
Processing result 52 for compound MK-2866
Finished result 51
Processing result 52 for compound MK-2866
Finished result 52
Processing result 

125

In [9]:
# Save the docs list to a pickle file
with open(
    "summarized_tool_run_docs.pkl",
    "wb",
) as f:
    pickle.dump(docs, f)

print("Docs saved to summarized_tool_run_docs.pkl")

Docs saved to summarized_tool_run_docs.pkl


In [ ]:
embedder = dspy.Embedder("openai/text-embedding-3-small", dimensions=512)
search = dspy.retrievers.Embeddings(
    embedder=embedder, corpus=docs, k=1
)  # FAISS auto-handled if installed


# 2) tiny RAG over memory
class MemoryRAG(dspy.Module):
    def __init__(self):
        self.respond = dspy.Predict("context, query -> answer: str")

    def forward(self, query):
        ctx = search(query).passages
        numbered_ctx = [f"Context {i + 1}:\n{passage}" for i, passage in enumerate(ctx)]
        return self.respond(context="\n\n".join(numbered_ctx), query=query)


rag = MemoryRAG()

# 3) example query: retrieve prior runs to estimate efficacy
q = "HSP90 inhibitor efficacy"
print(rag(query=q).answer)

In [ ]:
dspy.inspect_history()